In [2]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
#pd.options.mode.chained_assignment = None  # default='warn'


In [3]:
headerString = "date time time-taken c-ip cs-username cs-auth-group x-exception-id sc-filter-result cs-categories cs(Referer) sc-status s-action cs-method rs(Content-Type) cs-uri-scheme cs-host cs-uri-port cs-uri-path cs-uri-query cs-uri-extension cs(User-Agent) s-ip sc-bytes cs-bytes x-virus-id"
header=headerString.split(" ")
raw_data = pd.read_csv('./Data/message.txt', delimiter="\s+", index_col=False, encoding="utf-8", comment='#',names=header)


C:\Users\VRUma\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (10,16,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
df1 = raw_data.sample(frac=0.5,random_state=200) 

df1 = raw_data.sample(frac=0.5,random_state=200) 

df1=df1.drop(columns=['cs-username','cs-auth-group','date',
                      'time-taken','cs-method','cs-username','cs-auth-group','x-exception-id',
                      'sc-filter-result','cs-categories','s-action',
                      'rs(Content-Type)','cs(User-Agent)','s-ip','x-virus-id'])
df1

,time,c-ip,cs(Referer),sc-status,cs-uri-scheme,cs-host,cs-uri-port,cs-uri-path,cs-uri-query,cs-uri-extension,sc-bytes,cs-bytes
715527,21:09:25,922e695d8efdd9c2,http://porn984.xvideos.com/videos/flv/a/b/0/,400,http,porn984.xvideos.com,80,/videos/flv/a/b/0/xvideos.com_ab04b9e0c7680d89...,?e=1311256792&ri=1024&rs=85&h=af94236c25fd9e70...,flv;v=1,221,680
751099,21:11:08,6f977f9769f73681,http://www.manjam.com/play/living.aspx?page=8,200,http,bid.openx.net,80,/json,?c=OXM_23710861784&pid=b7e1fb6a-0828-46ce-8a50...,-,411,801
104380,20:39:42,fac7d857d7b69c83,"http://46.229.160.7/key=RCOVrmXbYZM,end=131128...",403,http,46.229.160.7,80,"/key=RCOVrmXbYZM,end=1311282649/data=480351662...",-,flv,322,439
185664,20:43:22,6d98469a3f1de6f4,http://urgecash.net/tgp/bfd/item/barefoot-domi...,503,http,urgecash.com,80,/images/download-tgp.png,-,png,935,465
140110,20:41:20,48fd5c69d3cfc0af,http://static.xhamster.com/xplayer17.swf,403,http,88.208.24.132,80,"/key=GUnBrc.cmPI,end=1311337882/data=480312684...",-,flv,724,317
...,...,...,...,...,...,...,...,...,...,...,...,...
509646,20:59:12,1f494a9319d17c11,http://translate.google.com/?hl=ar&tab=wT,204,http,csi.gstatic.com,80,/csi,?v=3&s=translate&action=at&it=st.73&tran=0&sl=...,-,239,468
675591,21:07:28,ce4f02e53b9760d5,http://static.xvideos.com/swf/xv-player.swf,400,http,porn784.xvideos.com,80,/videos/flv/9/d/a/xvideos.com_9dad74920f02d12f...,?e=1311350247&ri=1024&rs=85&h=f6a92e61e9f4971c...,flv,221,403
85302,20:38:50,da6476b106ecb084,http://medleyads.com/spot/5232.html,200,http,medleyads.com,80,/mad_history,?adgroups=931,-,1155,598
859070,21:16:25,a49359d7cd768c3b,http://x42.iloveim.com/build_3.9.2.1/comet.html,200,http,x42.iloveim.com,80,/servlets/events,?1311369374861,-,299,1183


In [81]:
from urllib.parse import urlparse

def preprocessing(df):
    df.replace("-", "",inplace = True)
    


In [88]:
def parse_url(data,column_toparse,scheme=True,domain=True,path=True,params=False,query=False,fragment=False):
    if column_toparse not in data.columns:
        print("Column name not found")
        return 0
    else:
        data['referer-protocol'],data['referer-domain'],data['referer-path'],data['referer-params'],data['referer-query'],data['referer-fragment'] = zip(*data[column_toparse].map(urlparse))
        choices=[scheme,domain,path,params,query,fragment]
        names=['referer-protocol','referer-domain','referer-path','referer-params','referer-query','referer-fragment']
        indexes=np.where(choices)[0]
        keep=[]
        for k in indexes:
            keep.append(names[k])
        dropped=list(set(names) - set(keep))
        data=data.drop(columns=dropped)
    return data
        
        
        
        
    
    


In [118]:
df=df1.sample(frac=1,random_state=1)
data=parse_url(df,'cs(Referer)')

preprocessing(data)
data

,time,c-ip,cs(Referer),sc-status,cs-uri-scheme,cs-host,cs-uri-port,cs-uri-path,cs-uri-query,cs-uri-extension,sc-bytes,cs-bytes,referer-protocol,referer-domain,referer-path
561329,21:01:45,593910c52e601ad6,http://www.mylovedboobs.com/archives/big-tits-...,200,http,www.mylovedboobs.com,80,/t/110712/cb0d592d70.jpg,,jpg,13071,613,http,www.mylovedboobs.com,/archives/big-tits-archive-4.html
515571,20:59:29,6d98469a3f1de6f4,http://video.xnxx.com/video88084/maya_gold_you...,304,http,static.xvideos.com,80,/css/comments_xnxx.css,,css,217,459,http,video.xnxx.com,/video88084/maya_gold_young_slut
190520,20:43:35,0e8fe56c260eb807,http://www.aflamk1.net/cat-10-p120,304,http,www.aflamk1.net,80,/style/elg/css/style.css,,css,420,691,http,www.aflamk1.net,/cat-10-p120
357254,20:51:30,48fd5c69d3cfc0af,http://static.xhamster.com/xplayer17.swf,403,http,88.208.24.131,80,"/key=io8g3-zl3cM,end=1311337549/data=480312379...",,flv,724,319,http,static.xhamster.com,/xplayer17.swf
420717,20:54:46,48fd5c69d3cfc0af,http://static.xhamster.com/xplayer17.swf,403,http,88.208.24.132,80,"/key=GUnBrc.cmPI,end=1311337882/data=480312684...",,flv,724,316,http,static.xhamster.com,/xplayer17.swf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416162,20:54:32,a8634e9101ab86e8,http://milfdreamtube.com/,200,http,www.milfdreamtube.com,80,/m/000/000/588/t1031x0e9825.jpg,,jpg,10333,387,http,milfdreamtube.com,/
648176,21:06:07,7c9b187f6042746d,http://www.4shared.com/,200,http,dc413.4shared.com,80,/network/search-suggest.jsp,?search=SUYgV0FTIEdPRCBBIFdPTUVO&format=jsonp,jsp,241,696,http,www.4shared.com,/
334406,20:50:23,9fec37a089918d52,,200,http,207.46.125.85,80,/gateway/gateway.dll,?Action=poll&LifeSpan=60&SessionID=264471285.2...,dll,621,442,,,
851673,21:16:04,d95137008ba37d77,http://sexarb.free.fr/vb/showthread.php?t=242,200,http,sexarb.free.fr,80,/vb/images/icons/000exclusive[1][1].gif,,gif,2325,529,http,sexarb.free.fr,/vb/showthread.php


In [119]:
def preprocessing(df):
    df.replace("-", "",inplace = True)
    
preprocessing(data)

In [124]:
def find_same_referer(data):
    data.loc[data['cs-host']==data['referer-domain'],'stayed']=1
    data['stayed'] = data['stayed'].fillna(0)
    return data



In [133]:
data.groupby('cs-uri-extension')

TypeError: Cannot use method 'nlargest' with dtype object

In [96]:
try:
    socket.inet_aton(addr)
except socket.error:
    print("OUI")

OUI


In [135]:
data.loc[data['cs-uri-path']=="/" & ]

,time,c-ip,cs(Referer),sc-status,cs-uri-scheme,cs-host,cs-uri-port,cs-uri-path,cs-uri-query,cs-uri-extension,sc-bytes,cs-bytes,referer-protocol,referer-domain,referer-path,stayed
449931,20:56:11,0103f1630ad2a135,,200,http,www.wikipedia.org,80,/,,,49131,89,,,,0.0
316140,20:49:32,ac7b7f9b79ea4dc8,,400,,,0,/,,,842,73,,,,1.0
718118,21:09:33,3c066977a5fdc3ca,,0,,,0,/,,,0,69,,,,1.0
697494,21:08:33,563e2aa90c6cfd3f,,200,http,currentip.net,80,/,,,1331,219,,,,0.0
456916,20:56:33,1e5b709d1918c5ce,,0,,,0,/,,,0,38,,,,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909843,21:18:54,5595bce6a07a2cf9,,0,,,0,/,,,0,61,,,,1.0
648799,21:06:08,aa977e319d0cf303,,0,,,0,/,,,0,51,,,,1.0
891311,21:17:57,4e870e89dcda8560,,0,,,0,/,,,0,40,,,,1.0
688271,21:08:06,74271c516552fe75,,0,,,0,/,,,0,52,,,,1.0
